# Introduction to Data Science 
# Lecture 9: Linear Regression 1
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

In this lecture, we'll discuss:
* Simple linear regression (SLR)
* Example: advertisement effects
* Descriptive vs. inferential viewpoints 
* Multiple linear regression 
* Nonlinear relationships 

Recommended reading:
* G. James, D. Witten, T. Hastie, and R. Tibshirani, An Introduction to Statistical Learning, Ch. 3 [digital version available here](http://www-bcf.usc.edu/~gareth/ISL/)


## Linear Regression 

*Linear regression* models the relationship between an independent (explanatory) variable $X$ and a (real-valued) dependent value $Y$.

#### Examples:
<table style="width:60%">
  <tr>
    <th>explanatory variable, X</th>
    <th>dependent variable, Y</th> 
  </tr>
  <tr>
    <td>square footage</td>
    <td>house price</td> 
  </tr>
  <tr>
    <td>advertising dollars spent</td>
    <td>profit</td> 
  </tr>
  <tr>
    <td>stress</td>
    <td>lifespan</td> 
  </tr>
  <tr>
    <td>?</td>
    <td>?</td> 
  </tr>  
</table>

<br> 

#### What is wrong with this? 

<table style="width:60%">
  <tr>
    <th>explanatory variable, X</th>
    <th>dependent variable, Y</th> 
  </tr>
  <tr>
    <td> time spent studying </td>
    <td> pass/fail </td> 
  </tr>
</table>


## Simple Linear Regression (SLR)

**Data**: We have $n$ samples $(x_1, y_1), \ (x_2, y_2),\ldots,(x_n, y_n)$.

**Model**: $y \sim \beta_0 + \beta_1 x$ 

**Goal**: Find the best values of $\beta_0$ and $\beta_1$, denoted $\hat{\beta}_0$ and $\hat{\beta}_1$, so that the prediction $y = \hat{\beta}_0 + \hat{\beta}_1 x$ "best fits" the data.

<img src="438px-Linear_regression.png" width="40%" alt="https://en.wikipedia.org/wiki/Linear_regression">

**Theorem.** 
The best parameters in the *least squares sense* are:
$$
\hat{\beta}_1 = \frac{\sum_{i=1}^n (x_i - \overline{x})(y_i - \overline{y}) }{\sum_{i=1}^n (x_i - \overline{x})^2}
\qquad \textrm{and} \qquad
\hat{\beta}_0 = \overline{y} -  \hat{\beta}_1 \overline{x}. 
$$
where $\overline{x} = \frac{1}{n} \sum_{i=1}^n x_i$ and $\overline{y} = \frac{1}{n} \sum_{i=1}^n y_i$. 

**Proof.** On board in class; see SLR.pdf.


## Simple linear regression with python

### Python packages for regression
There are several different python packages that do regression:
1. [statsmodels](http://statsmodels.sourceforge.net/)
+ [scikit-learn](http://scikit-learn.org/)
+ [SciPy](http://www.scipy.org/)
+ ... 


Today, we'll look at both `statsmodels` and `scikit-learn`. One can also use SciPy for linear regression, but its built-in functionality is comparatively limited. 


### Example dataset
To illustrate linear regression, we'll use the 'Advertising' dataset from
[here](http://www-bcf.usc.edu/~gareth/ISL/data.html)


For 200 different 'markets' (think different cities), this dataset consists of the number of sales of a particular product as well as the advertising budget for different media: TV, radio, and newspaper. 

We’ll use linear regression to study the effect of advertising on sales. 
Here, sales is the dependent variable and the budgets are the independent variables. This might help inform or evaluate an advertising strategy for this product.  

In [ ]:
# imports and setup

import scipy as sc
from scipy.stats import norm

import pandas as pd
import statsmodels.formula.api as sm
from sklearn import linear_model

import matplotlib.pyplot as plt
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6)

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
advert = pd.read_csv('Advertising.csv',index_col=0) #load data
advert

## Plot and describe the data

In [ ]:
plt.scatter(x=advert['TV'],y=advert['Sales'],c='r',marker='s',label='TV')
plt.scatter(x=advert['Radio'],y=advert['Sales'],c='b',marker='o',label='Radio')
plt.scatter(x=advert['Newspaper'],y=advert['Sales'],c='k',marker='*',label='Newspaper')

plt.legend(numpoints=1,loc=4)
plt.xlabel('Ad budget (Thousands of dollars)')
plt.ylabel('Sales (units of product)')
plt.show()

In [ ]:
advert.describe()

## Observations 
1. From the plot, it is clear that there is a relationship between the advertising budgets and sales. Basically, the more money spent, the larger the number of sales. 
+  The most money was spent on TV advertising. The amount for Radio and Newspaper is about the same in all markets, whereas the standard deviation for TV advertising is larger. 

## Questions
1. How can we quantify the relationship between advertising and sales? Can we predict the effect of each ad media on sales? Is the relationship linear? 
+  Which of the different ad media (TV, Radio, Newspaper) are the most effective at generating sales? 
+  Are there interactions between the different ad media?

First, let's just look at the **effect of TV advertising on sales**. We use the linear regression model
$$
Sales = \beta_0 + \beta_1 * TV.
$$

In [ ]:
ad_TV_ols = sm.ols(formula="Sales ~ TV", data=advert).fit()
ad_TV_ols.summary()

In [ ]:
plt.scatter(x=advert['TV'],y=advert['Sales'],c='k',marker='*',label='TV')
plt.plot(advert['TV'],ad_TV_ols.predict(),'k',color='blue',linewidth=3)

plt.xlabel('TV budget (Thousands of dollars)')
plt.ylabel('Sales (Thousand units of product)')
plt.show()

## Interpretation and discussion

The intercept of the line is $\hat{\beta}_0 = 7.032$. This means that without any TV advertising, the model predicts that 7,032 units of product will be sold. 

The slope of the line is $\hat{\beta}_1 = 0.0475$. This means that the model predicts that for every additional $1k spent on TV advertising, an additional 47.5 units of product are sold. 

**So how good is this fit?** 

One way to measure the quality of the fit is to look at the sum of the squared residuals,
$$
SSR = \sum_{i=1}^n (y_i - \hat{\beta}_0 - \hat{\beta}_1 x_i)^2. 
$$
Remember that SSR is the quantity that we minimized to find $\hat{\beta}_0$ and $\hat{\beta}_1$ in the first place (We called it $J(\beta_0,\beta_1)$.) If this number is very small, then the model fits the data very well. 


In [ ]:
ad_TV_ols.ssr

But how small is small?...good question. This number, SSR, is difficult to interpret by itself. A more easily interpretable number is the **$R^2$ value**. 

The $R^2$ value is an alternative way to measure how good of a fit the model is to the data. The benefit of the $R^2$ value over the SSR is that it is a proportion (takes values between 0 and 1) so it is easier to interpret what a *good* value is. We first define the residual sum of squares (SSR) and total sum of squares (TSS) by
$$
SSR = \sum_{i=1}^n (y_i - \hat\beta_0 - \hat\beta_1 x_i)^2
\qquad \text{and} \qquad 
TSS = \sum_{i=1}^n (y_i - \bar y)^2. 
$$
SSR measures the amount of variability left unexplained after the linear regression. TSS measures the total variance in the data. We compute the $R^2$ value as
$$
R^2 = 1 - \frac{SSR}{TSS}.
$$
This is the proportion of the variance explained by the model. A model is good if the $R^2$ value is nearly one (the model explains all of the variance in the data). 

In our model, the value is $R^2 = 0.612$, which isn't bad. The model explains $61\%$ of the variability in sales. 

*Note*: for linear regression, the $R^2$ value is the same as correlation, but the $R^2$ value more easily generalizes to more complicated regression models than correlation, so the $R^2$ value is typically considered instead of correlation.

![image](http://imgs.xkcd.com/comics/linear_regression.png)



## Repeating the simple linear regression with scikit-learn

In [ ]:
lr = linear_model.LinearRegression() # create a linear regression object

# scikit-learn doesn't work as well with pandas, so we have to extract values 
x = advert['TV'].values.reshape(advert['TV'].shape[0],1)
y = advert['Sales'].values.reshape(advert['Sales'].shape[0],1)

lr.fit(X=x, y=y)

plt.scatter(x, y,  color='black')
plt.plot(x, lr.predict(x), color='blue', linewidth=3)

plt.xlabel('TV budget (Thousands of dollars)')
plt.ylabel('Sales (Thousand units of product)')
plt.show()

## Hypothesis testing in linear regression

In lecture 4, we introduced a distinction between descriptive statistics and statistical inference. 
1. In descriptive statistics, one seeks just to describe the data. In the present setting, we have described how the response variable linearly depends on the predictor variable by minimizing the residual sum of squares (RSS). 
+ The statistical inference way of looking at this problem would be to suppose that there exists a ground truth population with $x$ and $y$ related by 
$$
y = \beta_0 + \beta_1 x 
$$
for some unknown values of $\beta_0$ and $\beta_1$. 
Our sampled data consists of points $(x_i, y_i)$ of the form 
$$
y_i = \beta_0 + \beta_1 x_i + \varepsilon_i. 
$$
Here $\varepsilon_i$ are random variable (say normally distributed) that we think of as "error" being introduced into the samples. The job of the statistician is to *infer* the values of $\beta_0$ and $\beta_1$ from the erroneous data.  

This is precisely the setting we were in when determining whether a coin was fair. There, we had a sample proportion of heads (analogous to the samples $(x_i,y_i)$ here.) We used the Central Limit Theorem to say that the variance (standard error) is given by
$$
SE(\hat \mu)^2 = \sigma^2/n 
$$
Using this value and assuming the null hypothesis (the coin is fair), we could evaluate the likelihood of obtaining a sample as extreme as the one obtained. 

In simple linear regression, we will take the null hypothesis to be
$$
H_0: \text{There is no relationship between $x$ and $y$} \iff \beta_1 = 0 
$$
with alternative
$$
H_a: \text{There is a relationship between $x$ and $y$}  \iff \beta_1 \neq 0 
$$
We assume that $\varepsilon$ is a normal random variable with zero mean and variance $\sigma^2$. Using similar ideas as above, the standard error for $\hat \beta_0$ and $\hat \beta_1$ (estimates of true parameters in this model) are computed to be 
$$
SE(\hat \beta_0)^2 = \sigma^2 \left( \frac{1}{n} + \frac{\bar x^2}{\sum_{i=1}^n (x_i - \bar x)^2} \right) 
\quad \text{and} \quad
SE(\hat \beta_1)^2 =  \frac{\sigma^2}{\sum_{i=1}^n (x_i - \bar x)^2}
$$


For this hypothesis test, the test statistic is 
$$
t = \frac{ \hat \beta_1 - 0}{SE(\hat \beta_1)},
$$
which under the assumptions of the null hypothesis, is distributed according to the $t$ distribution with $n-2$ degrees of freedom. The $p$-value is computed as the probability of observing a value as extreme as $|t|$. A small $p$-value is interpreted to mean that there is an association between the independent and dependent variables. 

### An experiment
Before we go back and discuss the $p$-values for the advertising data, let's look at some synthetic data. We generate 100 random points according the model 
$$
y = 3*x + \varepsilon,
$$
where $\varepsilon$ is normally distributed with mean zero and standard deviation 2. The best fit is found and this process is repeated 1000 times. 

In [ ]:
f= lambda x: 3*x
x = sc.linspace(-2,2,3).reshape(3,1)
plt.figure(0)

sample_size = 100
betaOnes = []
for ii in range(1000):
    xp = norm.rvs(size=sample_size)
    yp = f(xp)+norm.rvs(size=sample_size,scale=2)
    
    if ii is 1: plt.plot(xp,yp,'ro')        

    lr = linear_model.LinearRegression()
    lr.fit(X=xp.reshape(100,1), y=yp)
    plt.plot(x,lr.predict(x),color='blue',linewidth=1)
    
    # Collect the slopes 
    betaOnes.append(lr.coef_[0])

plt.plot(xp,f(xp),'k',linewidth=3)
plt.xlim(-2,2)    
plt.ylim(-10,10)    
plt.title('Best fit lines for points generated from a random model')
plt.show()

plt.figure(1)
plt.hist(betaOnes)
plt.title('A histogram of estimates for beta_1')
plt.show()


Now, let's return to the linear regression model of sales on TV advertising, 
$$
Sales = \beta_0 + \beta_1 * TV.
$$
Looking at the `statsmodels` output, we see that the $p$-values for both the intercept and the slope are very small. The probability of seeing obtaining these values is nearly zero, assuming $H_0$ is true. So, we reject the null hypothesis and say there is an association between TV advertising (independent variable) and sales (dependent variable). 


## Other advertisement methods?  
Recall that we not only know the ad budget for TV, but also Radio and Newspaper. 

Next, let's repeat the linear regression analysis for the other types of advertisements using `statsmodels`. 

In [ ]:
ad_Radio_ols = sm.ols(formula="Sales ~ Radio", data=advert).fit()
ad_Radio_ols.summary()

In [ ]:
ad_Newspaper_ols = sm.ols(formula="Sales ~ Newspaper", data=advert).fit()
ad_Newspaper_ols.summary()

In [ ]:
plt.scatter(x=advert['TV'],y=advert['Sales'],c='r',marker='s',label='TV')
plt.scatter(x=advert['Radio'],y=advert['Sales'],c='b',marker='o',label='Radio')
plt.scatter(x=advert['Newspaper'],y=advert['Sales'],c='k',marker='*',label='Newspaper')
plt.legend(numpoints=1,loc=4)

plt.plot(advert['TV'],ad_TV_ols.predict(),c='r',linewidth=3)
plt.plot(advert['Radio'],ad_Radio_ols.predict(),c='b',linewidth=3)
plt.plot(advert['Newspaper'],ad_Newspaper_ols.predict(),c='k',linewidth=3)

plt.xlabel('Ad budget (Thousands of dollars)')
plt.ylabel('Sales (units of product)')
plt.show()


## Interpretation

*So what is the most effective advertising media?*

The slope for radio is largest, so you might argue that this is the most effective advertising media. For every additional \$1k spent on Radio advertising, an additional 202 units of product are sold. (Compare to 54.7 for newspaper and 47.5 for TV.)

On the other hand, the $R^2$ value for radio is just $33\%$. So the model isn't explaining as much of the data as the model for TV advertising ($R^2 = 61\%$), but is explaining more than the model for newspaper advertising ($R^2 = 5\%$). 

The main problem with the approach here is that for each advertising media we look at, we're ignoring the ads in the other media. For example, in the model for TV advertising, 
$$
Sales = \beta_0 + \beta_1 * TV,
$$
we're ignoring both Radio and Newspaper advertising. 

We need to take all three into account at once. Maybe we can construct a model that looks like 
$$
Sales = \beta_0 + \beta_1 * TV + \beta_2*Radio + \beta_3*Newspaper. 
$$
This is the idea behind Multiple Linear Regression. 

## Multiple Linear Regression

**Model:**
$$
Sales = \beta_0 + \beta_1 * TV + \beta_2*Radio + \beta_3*Newspaper. 
$$


In [ ]:
ad_all_ols = sm.ols(formula="Sales ~ TV + Radio + Newspaper", data=advert).fit()
ad_all_ols.summary()

## Interpretation

Spending an additional \$1,000  on radio advertising results in an increase in sales by 189 units. Radio is the most effective at method of advertising. 

In multilinear regression, the F-test is a way to test the null hypothesis,
$$
H_0 = \textrm{all coefficients are zero.}
$$
In this case, we see that the $p$-value for the $F$-statistic is vanishingly small - indicating that our model is significant. 

Now let's consider the individual coefficients in the model and their $p$-values. Note that the coefficients for TV and Radio are approximately the same as for simple linear regression. The coefficient for Newspaper changed significantly. Furthermore, note that the $p$-value is now very large $p=0.86$. There is not sufficient evidence to reject the null hypothesis that the Newspaper and Sales variables have no relationship. 

So then why did the simple linear regression give that there is a relationship between Newspaper and Sales Variables? 
*Newspaper is actually a confounder!* (Remember the example where temperature is a confounder for pool deaths and ice creams sales.) Let's look at the correlations between the four variables. Recall that correlation between two variables is given by 
$$
r_{x,y} = \frac{ \frac{1}{n}\sum_{i=1}^n (x_i - \bar x) (y_i - \bar y)}{s_x s_y}.
$$
Correlation is a number between −1 to +1 and measures how much the two variables vary together. 

Plotted below is also a scatter plot matrix, which is a good way of visualizing the correlations. 

In [ ]:
print(advert.corr())
pd.plotting.scatter_matrix(advert, figsize=(10, 10), diagonal='kde')
plt.show()

The correlation between Newspaper and Radio is 0.35, which implies that in markets where the company advertised using Radio, they also advertised using newspaper. Thus, the influence of Radio on Sales can be incorrectly attributed to Newspaper advertisements! 

This leads us to the following linear regression model, where we forget about Newspaper advertisements:
$$
\text{Sales} = \beta_0 + \beta_1 * \text{TV_budget} + \beta_2*\text{Radio_budget} 
$$


In [ ]:
ad_TR_ols = sm.ols(formula="Sales ~ TV + Radio", data=advert).fit()
ad_TR_ols.summary()

This model performs pretty well. It accounts for $R^2 = 90\%$ of the variance in the data. 

In [ ]:
plt.rcParams['figure.figsize'] = (15, 9)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(xs=advert['TV'], ys=advert['Radio'], zs=advert['Sales'])

x = sc.linspace(advert['TV'].min(), advert['TV'].max(), 100)
y = sc.linspace(advert['Radio'].min(), advert['Radio'].max(), 100)
X,Y = sc.meshgrid(x,y)
par = dict(ad_TR_ols.params)
Z = par["Intercept"] + par["TV"]*X + par["Radio"]*Y 
surf = ax.plot_surface(X, Y, Z,cmap=cm.Greys, alpha=0.2)

ax.view_init(25,-71)

ax.set_xlabel('TV budget')
ax.set_ylabel('Radio budget')
ax.set_zlabel('Sales')

plt.show()

## Nonlinear relationships

We can consider the interaction between TV and Radio advertising in the model, by taking 
$$
\text{Sales} = \beta_0 + \beta_1 * \text{TV_budget} + \beta_2*\text{Radio_budget} + \beta_3 \text{TV_budget} *\text{Radio_budget}. 
$$
The rational behind the last term is that perhaps spending $x$ on television advertising and $y$ on radio advertising leads to more sales than simply $x+y$. In marketing this is known as the *synergy effect* and in statistics it is known as the *interaction effect*.

**Note**: even though the relationship between the independent and dependent variables is nonlinear, the model is still linear. 

In [ ]:
ad_NL = sm.ols(formula="Sales ~ TV + Radio + TV*Radio", data=advert).fit()
ad_NL.summary()

This model is really excellent. All of the $p$-values are small and $R^2 = 97\%$ of the variability in the data is accounted for by the model. 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(xs=advert['TV'], ys=advert['Radio'], zs=advert['Sales'])

x = sc.linspace(advert['TV'].min(), advert['TV'].max(), 100)
y = sc.linspace(advert['Radio'].min(), advert['Radio'].max(), 100)
X,Y = sc.meshgrid(x,y)
par = dict(ad_NL.params)
Z = par["Intercept"] + par["TV"]*X + par["Radio"]*Y + par["TV:Radio"]*X*Y
surf = ax.plot_surface(X, Y, Z,cmap=cm.Greys, alpha=0.2)

ax.view_init(25,-71)

ax.set_xlabel('TV budget')
ax.set_ylabel('Radio budget')
ax.set_zlabel('Sales')

plt.show()

## A word of caution on overfitting (more on this later)

It is tempting to include a lot of terms in the regression, but this is problematic. A useful model will  *generalize* beyond the data given to it. 

![image](http://imgs.xkcd.com/comics/curve_fitting_2x.png)
